In [49]:
import os
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
from skimage.measure import label, regionprops, find_contours
import matplotlib.pyplot as plt

In [77]:
imagesPrefix = "../../ADEChallengeData2016/images/training"
masksPrefix = "../../ADEChallengeData2016/annotations/training"
images = os.listdir(imagesPrefix)
masks = os.listdir(masksPrefix) 
print(len(images))
print(len(masks))

20210
20210


In [78]:
images = sorted(images)
masks = sorted(masks)
images_path = [imagesPrefix + "/" + image for image in images]
masks_path = [masksPrefix + "/" + mask for mask in masks]

In [79]:
def mask_to_border(mask):
    h, w = mask.shape
    border = np.zeros((h, w))

    contours = find_contours(mask, 128)
    for contour in contours:
        for c in contour:
            x = int(c[0])
            y = int(c[1])
            border[x][y] = 255

    return border

""" Mask to bounding boxes """
def mask_to_bbox(mask):
    bboxes = []

    mask = mask_to_border(mask)
    lbl = label(mask)
    props = regionprops(lbl)
    for prop in props:
        x1 = prop.bbox[1]
        y1 = prop.bbox[0]

        x2 = prop.bbox[3]
        y2 = prop.bbox[2]

        bboxes.append([x1, y1, x2, y2])

    return bboxes

def parse_mask(mask):
    mask = np.expand_dims(mask, axis=-1)
    mask = np.concatenate([mask, mask, mask], axis=-1)
    return mask


def clearing_masks(mask):
    mask[mask != 13] = 0
    mask[mask == 13] = 255
    return mask

In [83]:
annotation = {
    'images': [],
    'annotations': []
}

for i, (image, mask, image_path, mask_path) in tqdm(enumerate(zip(images, masks, images_path, masks_path))):

    img = cv2.imread(image_path, cv2.IMREAD_COLOR)
    msk = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    annotation['images'].append(
        {
            'file_name': image,
            'height': img.shape[0],
            'width': img.shape[1],
            'id': i
        })
    print(annotation['images'])

    msk = clearing_masks(msk)

    bboxes = mask_to_bbox(msk)

    for bbox in bboxes:
        x = cv2.rectangle(x, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (255, 0, 0), 2)

    cat_image = np.concatenate([x, parse_mask(y)], axis=1)




        to_dump['annotations'].append(
                    {
                        'segmentation': seg,
                        'area': area,
                        'iscrowd': 0,
                        'image_id': i,
                        'bbox': bbox,
                        'category_id': category_id,
                        'id': instance_counter,
                        'ignore': 0
                    }
                )

    plt.imshow(cat_image)
    print(bboxes)
    #cv2.imwrite(f"results/{name}.png", cat_image)
    break


0it [00:00, ?it/s]

[{'file_name': 'ADE_train_00000001.jpg', 'height': 512, 'width': 683, 'id': 0}]


'\n    clearing_masks(y)\n\n    bboxes = mask_to_bbox(y)\n\n    for bbox in bboxes:\n        x = cv2.rectangle(x, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (255, 0, 0), 2)\n\n    cat_image = np.concatenate([x, parse_mask(y)], axis=1)\n\n\n\n\n        to_dump[\'annotations\'].append(\n                    {\n                        \'segmentation\': seg,\n                        \'area\': area,\n                        \'iscrowd\': 0,\n                        \'image_id\': i,\n                        \'bbox\': bbox,\n                        \'category_id\': category_id,\n                        \'id\': instance_counter,\n                        \'ignore\': 0\n                    }\n                )\n\n    plt.imshow(cat_image)\n    print(bboxes)\n    #cv2.imwrite(f"results/{name}.png", cat_image)\n    break\n'

In [48]:
for x, y in zip(images, masks):
    print(x)
    print(y)
    x = cv2.imread(x, cv2.IMREAD_COLOR)
    y = cv2.imread(y, cv2.IMREAD_GRAYSCALE)
    print(x)
    print(y)
    break

../../ADEChallengeData2016/images/training/ADE_train_00000001.jpg
../../ADEChallengeData2016/annotations/training/ADE_train_00000001.png
[[[187 187 181]
  [187 187 181]
  [187 187 181]
  ...
  [179 179 173]
  [179 179 173]
  [179 179 173]]

 [[187 187 181]
  [187 187 181]
  [187 187 181]
  ...
  [178 178 172]
  [178 178 172]
  [178 178 172]]

 [[187 187 181]
  [187 187 181]
  [187 187 181]
  ...
  [178 178 172]
  [178 178 172]
  [178 178 172]]

 ...

 [[ 66  81  77]
  [ 66  81  77]
  [ 67  82  78]
  ...
  [ 55  49  42]
  [ 55  49  42]
  [ 55  49  42]]

 [[ 70  84  80]
  [ 70  84  80]
  [ 68  82  78]
  ...
  [ 52  46  39]
  [ 52  46  39]
  [ 52  46  39]]

 [[ 71  85  81]
  [ 69  83  79]
  [ 68  79  76]
  ...
  [ 53  47  40]
  [ 53  47  40]
  [ 53  47  40]]]
[[0 0 0 ... 6 6 6]
 [0 0 6 ... 6 6 6]
 [0 0 6 ... 6 6 6]
 ...
 [4 4 4 ... 1 1 0]
 [4 4 4 ... 1 1 0]
 [4 4 4 ... 1 1 1]]


In [39]:
len(images)

20210